In [1]:
from astroquery.mast import Observations

In [3]:
%%time
print("Finding Observations for ACS/WFC images...")
obs_table = Observations.query_criteria(objectname="NGC0628", obs_collection="HST",
                                       filters=["F435W", "F555W", "F658N", "F606W", "F814W", "detection"],
                                        proposal_id=[10402, 15645, 9796, 13804, 11675],
                                       instrument_name="ACS/WFC")
print("Done")
# do not use detection and N filters

Finding Observations for ACS/WFC images...
Done
CPU times: user 48 ms, sys: 9.51 ms, total: 57.5 ms
Wall time: 1min 5s


In [12]:
print("Downloading data.\n This may take several minutes... \n")
products = Observations.get_product_list(obs_table)
data_prod = ['FLC']       # other options are ['FLT','DRC','DRZ']
data_type = ['HAP-SVM']   # other options are ['CALACS','CALWF3','CALWP2']

Observations.download_products(products, project=data_type, 
                               productSubGroupDescription=data_prod)
print("Done")

 This may take several minutes... 

INFO: 114 of 172 products were duplicates. Only returning 58 unique product(s). [astroquery.mast.utils]
Done


In [7]:
import shutil
import glob
import os

# Gathering the FLC file names
input_flcs = glob.glob(os.path.join('mastDownload','HST','*','*.fits'))

# Moving files from mastDownload directory into current directory
for flc in input_flcs:
    shutil.copy(flc, os.path.basename(flc))

shutil.rmtree('mastDownload') # removes the extra directories
print("Done")

Done


In [9]:
from astropy.io import fits

flc_files = glob.glob("hst*_acs_wfc_*_j*_flc.fits")
flc_files.sort()

count = 0
for i,f in enumerate(flc_files):
    try: hdu = fits.open(f)
    except:
        print(f,"not good. Please remove.")
        count = 1 + count

print("Done")

Done


# Mosaicking 

Date: 23 April 2025

Now we embark on mosaicking. <br>
The first filter that I will be mosaicking with is f435w. For the first run, we want the `expand_refcat` and `writecat` set to `True`. This will create a catalog of stars against which the remaining fields can be matched. 

In [13]:
import glob
from drizzlepac import tweakreg

# Start with F435W, collecting all 435W files
flc_files = glob.glob("hst_*_*_acs_wfc_f435w_*_flc.fits")
flc_files.sort()            # Alphabetizing files
reffile = flc_files[0]      d# Select a reference file to use on all FITS

tweakreg.TweakReg(flc_files, refimage=reffile, 
                  imagefindcfg={'threshold':100, 'conv_width':3.5},
                  expand_refcat=True, writecat=True, 
                  enforce_user_order=False, 
                  fitgeometry='rscale', # allows shift, rotation & rescale
                  minobj=3, sigma=3.0, nclip=3, 
                  searchrad=1.0, searchunits='arcseconds', 
                  peakmax=70000, ylimit=0.3, interactive=False, 
                  shiftfile=True, reusename=True, 
                  outshifts='M74_shifts_acs_f435w.txt', 
                  updatehdr=False)



The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol


The first tweakreg started at 9:54 am.

Above is the error I got at 10:40 am.

To fix that, I will increase the threshold from 100 to 110. 

In [ ]:
import glob
from drizzlepac import tweakreg

# Start with F435W, collecting all 435W files
flc_files = glob.glob("hst_*_*_acs_wfc_f435w_*_flc.fits")
flc_files.sort()            # Alphabetizing files
reffile = flc_files[0]      d# Select a reference file to use on all FITS

refcat = "hst_9796_04_acs_wfc_f435w_j8ol04fr_flc_refxy_catalog.coo"
tweakreg.TweakReg(flc_files, refimage=reffile, 
                  imagefindcfg={'threshold':110, 'conv_width':3.5},
                  expand_refcat=False, writecat=False,  # do not need in this run, using refcat
                  refcat=refcat,
                  enforce_user_order=False, 
                  fitgeometry='rscale', # allows shift, rotation & rescale
                  minobj=3, sigma=3.0, nclip=3, 
                  searchrad=1.0, searchunits='arcseconds', 
                  peakmax=70000, ylimit=0.3, interactive=False, 
                  shiftfile=True, reusename=True, 
                  outshifts='M74_shifts_acs_f435w.txt', 
                  updatehdr=False)

The above did not work and instead gave me a bunch of warning. This is an example warning:

This run of tweakreg had created a shiftfile which contained this:

These nan values are telling me that something is wrong with this run so I am deleting this shiftfile. 

In [ ]:
import glob
from drizzlepac import tweakreg

# Start with F435W, collecting all 435W files
flc_files = glob.glob("hst_*_*_acs_wfc_f435w_*_flc.fits")
flc_files.sort()            # Alphabetizing files
reffile = flc_files[0]      d# Select a reference file to use on all FITS

refcat = "hst_9796_04_acs_wfc_f435w_j8ol04fr_flc_refxy_catalog.coo"
tweakreg.TweakReg(flc_files, refimage=reffile, 
                  imagefindcfg={'threshold':102, 'conv_width':3.5},
                  expand_refcat=False, writecat=False,  # do not need in this run, using refcat
                  refcat=refcat,
                  enforce_user_order=False, 
                  fitgeometry='rscale', # allows shift, rotation & rescale
                  minobj=3, sigma=3.0, nclip=3, 
                  searchrad=1.0, searchunits='arcseconds', 
                  peakmax=70000, ylimit=0.3, interactive=False, 
                  shiftfile=True, reusename=True, 
                  outshifts='M74_shifts_acs_f435w.txt', 
                  updatehdr=False)

I am getting the same errors. I will need to bring in more changes

In [ ]:
tweakreg.TweakReg(flc_files, refimage=reffile, 
                  imagefindcfg={'threshold':80, 'conv_width':3.5},
                  expand_refcat=False, writecat=False,  # do not need in this run, using refcat
                  refcat=refcat,
                  enforce_user_order=False, 
                  fitgeometry='rscale', # allows shift, rotation & rescale
                  minobj=3, sigma=2.5, nclip=3, 
                  searchrad=2.0, searchunits='arcseconds', 
                  peakmax=70000, ylimit=0.3, interactive=False, 
                  shiftfile=True, reusename=True, 
                  outshifts='M74_shifts_acs_f435w.txt', 
                  updatehdr=False)

This is not working either. 

`WARNING: No matches found within a search radius of 40 pixels.`

`Not enough matches (< 3) found for input image: hst_9796_04_acs_wfc_f435w_j8ol04fr_flc.fits`

In [ ]:
tweakreg.TweakReg(flc_files, refimage=reffile, 
                  imagefindcfg={'threshold':50, 'conv_width':3.5},
                  expand_refcat=False, writecat=False,  # do not need in this run, using refcat
                  refcat=refcat,
                  enforce_user_order=False, 
                  fitgeometry='rscale', # allows shift, rotation & rescale
                  minobj=2, sigma=2, nclip=3, 
                  searchrad=2.5, searchunits='arcseconds', 
                  peakmax=70000, ylimit=0.3, interactive=False, 
                  shiftfile=True, reusename=True, 
                  outshifts='M74_shifts_acs_f435w.txt', 
                  updatehdr=False)

This did not run either. Could not find 2 objects. 

In [ ]:
tweakreg.TweakReg(flc_files, refimage=reffile, 
                  imagefindcfg={'threshold':10, 'conv_width':3.5},
                  expand_refcat=False, writecat=False,  # do not need in this run, using refcat
                  refcat=refcat,
                  enforce_user_order=False, 
                  fitgeometry='rscale', # allows shift, rotation & rescale
                  minobj=2, sigma=1.0, nclip=3, 
                  searchrad=3, searchunits='arcseconds', 
                  peakmax=70000, ylimit=0.3, interactive=False, 
                  shiftfile=True, reusename=True, 
                  outshifts='M74_shifts_acs_f435w.txt', 
                  updatehdr=False)

did not work

Now, I am trying this with Pau's help. Let's see what happens. 

In [ ]:
tweakreg.TweakReg(flc_files, refimage=reffile, 
                  imagefindcfg={'threshold':30, 'conv_width':3.5},
                  expand_refcat=False, writecat=False, 
                  enforce_user_order=False, 
                  fitgeometry='rscale', # allows shift, rotation & rescale
                  minobj=1, sigma=0.5, nclip=1, 
                  searchrad=10.0, searchunits='arcseconds', 
                  peakmax=70000.0, ylimit=0.3, interactive=False, 
                  shiftfile=True, reusename=True, 
                  outshifts='M74_shifts_acs_f435w.txt', 
                  updatehdr=False)

In [1]:
from XRBID.AutoPhots import RunPhots